In [20]:
from google.colab import drive
drive.mount('/content/drive')


import pandas as pd
from transformers import (
    BertTokenizer,
    BertForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
import torch

!ls "/content/drive/My Drive/Colab_Notebooks/data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 agnostic_bert_article_embeddings.pkl	 datacluster_1_with_embeddings.pkl
 agnostic_bert_first100_embeddings.pkl	 datacluster_2_with_embeddings.pkl
 agnostic_bert_title_embeddings10k.pkl	 datacluster_3_with_embeddings.pkl
 agnostic_bert_title_embeddings.pkl	 datacluster_4_with_embeddings.pkl
 all-the-news-2-1.csv			 dbscan_Agnostic_clusters_analysis.csv
 bert_article_embeddings10K.pkl		 full_clusterd.pkl
 bert_article_embeddings.pkl		 full_clustered.pkl
 bert_first100_embeddings.pkl		 full_data_fixed_dates.pkl
 bert_title_embeddings10K.pkl		 full_dataset_Fine_embeddings.pkl
 cleaned_100k.csv			 full_dataset_with_embeddings.pkl
 cleaned_10k.csv			 full_data_with_dbscan.pkl
 cleaned_10k_with_first100.csv		 full_data_with_dbscan_revised.pkl
 cleaned_15k.csv			'full_data_with_finetuned_embeddings&dbscan.pkl'
 cleaned_30k.csv			 full_data_with_finetuned_embeddings.

In [16]:
df=pd.read_pickle('/content/drive/My Drive/Colab_Notebooks/data/full_clustered.pkl')
df.head()

,Unnamed: 0.1,Unnamed: 0,year,month,day,author,title,article,url,publication,F100,bert_embedding,bert_F100_embedding,agnostict_title_embedding,agnostic_bert_F100_embedding,cluster,ag_title_cluster
0,2489713,2489500,2019,11.0,29,"Marcos Sainz, Returnly CTO & CNBC Technology E...",4 ways to take the headache and hassle out of ...,"Returns can be a monumental headache, not only...",https://www.cnbc.com/2019/11/29/4-ways-to-take...,CNBC,"Returns can be a monumental headache, not only...","[-0.45904645, -0.1318392, -0.13017993, 0.24272...","[-0.45904645, -0.1318392, -0.13017993, 0.24272...","[0.010146772, 0.045087833, 0.05110123, 0.02414...","[-0.06112704, -0.04745357, 0.0007734501, 0.001...",0,0
1,2491648,2491435,2019,12.0,2,Mark Hay,Interview with a Couple Where One Partner Is H...,"For many, HIV is the ultimate boogeyman of the...",https://www.vice.com/en_us/article/pa7nw7/sero...,Vice,"For many, HIV is the ultimate boogeyman of the...","[-0.11113172, -0.72078353, -1.2201693, 0.06096...","[-0.11113172, -0.72078353, -1.2201693, 0.06096...","[-0.055526007, 0.13783266, -0.041953392, 0.003...","[-0.015344072, 0.05901287, -0.07621075, 0.0554...",0,0
2,2491649,2491436,2019,12.0,2,Thomas Lewton,"Psychedelics Are for Queer Folks, Too",When Dee Adams decided to take an Ayahuasca t...,https://www.vice.com/en_us/article/8xwz95/lgbt...,Vice,When Dee Adams decided to take an Ayahuasca tr...,"[0.1243837, -0.023502616, -0.28594422, -0.2027...","[0.1243837, -0.023502616, -0.28594422, -0.2027...","[0.028439248, -0.029162535, 0.0124535, 0.02126...","[0.0025998014, 0.04433202, 0.015134116, 0.0711...",0,0
3,2491650,2491437,2019,12.0,2,David Gilbert,Trump Won’t Take Part in This Week’s Landmark ...,President Donald Trump will not be attending t...,https://www.vice.com/en_us/article/43kjqw/trum...,Vice,President Donald Trump will not be attending t...,"[-0.13597916, 0.026650878, 0.0071579902, 0.059...","[-0.13597916, 0.026650878, 0.0071579902, 0.059...","[0.04152333, 0.002287072, 0.0862064, -0.063820...","[0.030706804, 0.0027535276, 0.05282167, -0.046...",2,2
4,2491651,2491438,2019,12.0,2,Lil' Deb's Oasis,Gluten Free Coconut Shrimp Recipe,Servings: 4Prep time: 15 minutesTotal time: 45...,https://www.vice.com/en_us/article/8xwpja/glut...,Vice,Servings: 4Prep time: 15 minutesTotal time: 45...,"[-0.6875796, -0.013337972, -0.08837862, 0.0419...","[-0.6875796, -0.013337972, -0.08837862, 0.0419...","[-0.05361938, 0.029068304, -0.052942466, 0.004...","[-0.05276871, 0.014421557, 0.022626739, -0.002...",5,5


In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00


#Fine tuning training function

In [24]:
import pandas as pd
from transformers import (
    BertTokenizer,
    BertForMaskedLM,
    BertModel,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
import torch
import numpy as np
from datasets import Dataset
from google.colab import drive
import os

drive.mount('/content/drive')

# נתיב בסיסי לשמירת המודלים
model_base_path = "/content/drive/My Drive/Colab_Notebooks/models/"

# פונקציה להפקת embedding מהמודל שעבר fine-tuning
def get_finetuned_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embedding

# הפונקציה המשולבת לאימון ולשמירת embeddings
def fine_tune_cluster(df, cluster_num, epochs=3):
    print(f"\n🚀 Fine-Tuning Cluster #{cluster_num}")

    # סינון הדאטה לפי קלאסטר ספציפי
    df_cluster = df[df['ag_title_cluster'] == cluster_num].copy()

    # יצירת רשימת טקסטים לאימון
    texts = df_cluster['F100'].tolist()

    # יצירת Dataset של HuggingFace
    dataset = Dataset.from_dict({"text": texts})

    # פיצול הדאטה לסט אימון (90%) וסט הערכה (10%)
    split_dataset = dataset.train_test_split(test_size=0.1)
    train_dataset = split_dataset['train']
    eval_dataset = split_dataset['test']

    # טעינת המודל והטוקנייזר של BERT
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')

    # פונקציה להמרת הטקסט לטוקנים
    def tokenize_function(example):
        return tokenizer(
            example['text'],
            padding='max_length',
            truncation=True,
            max_length=128
        )

    # המרת הדאטה לטוקנים (גם ל-train וגם ל-eval)
    tokenized_train = train_dataset.map(tokenize_function, batched=True)
    tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

    # יצירת DataCollator לאימון Masked Language Modeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15
    )

    # הגדרת הפרמטרים של האימון
    training_args = TrainingArguments(
        output_dir=f"{model_base_path}/fine_tuned_bert_cluster_{cluster_num}",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        num_train_epochs=epochs,
        weight_decay=0.01,
        per_device_train_batch_size=8,
        save_total_limit=2,
        logging_steps=50
    )

    # יצירת האובייקט Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        data_collator=data_collator
    )

    # הרצת האימון
    trainer.train()

    # שמירת המודל לאחר האימון בתיקייה המתאימה
    model_save_path = f"{model_base_path}/fine_tuned_bert_cluster_{cluster_num}"
    os.makedirs(model_save_path, exist_ok=True)  # יצירת התיקייה אם לא קיימת

    trainer.save_model(model_save_path)
    tokenizer.save_pretrained(model_save_path)

    print(f"\n✅ Cluster #{cluster_num} Fine-Tuned and Model Saved at {model_save_path}")

    # הפקת embeddings חדשים לכל הטקסטים בקלאסטר הנוכחי עם המודל החדש
    fine_tuned_model = BertModel.from_pretrained(model_save_path)

    embeddings_list = []
    for idx, text in enumerate(df_cluster['F100']):
        embedding = get_finetuned_embedding(text, fine_tuned_model, tokenizer)
        embeddings_list.append(embedding)
        if (idx + 1) % 100 == 0 or (idx + 1) == len(df_cluster):
            print(f"Processed {idx+1}/{len(df_cluster)} embeddings.")

    # שמירת ה-embeddings לעמודה החדשה
    df_cluster['F100_finetuning_embedding'] = embeddings_list

    # החזרת ה-DataFrame המעודכן
    return df_cluster

# טעינת הדאטה המקורי
df = pd.read_pickle('/content/drive/My Drive/Colab_Notebooks/data/full_data_with_finetuned_embeddings.pkl')

# יצירת DataFrame אחד גדול מכל הקלאסטרים
all_clusters_dfs = []

for cluster_num in df['ag_title_cluster'].unique():
    cluster_df = fine_tune_cluster(df, cluster_num, epochs=3)
    all_clusters_dfs.append(cluster_df)

# איחוד כל הקלאסטרים ל-DataFrame אחד
final_df = pd.concat(all_clusters_dfs, ignore_index=True)

# שמירת הקובץ המאוחד ל-Drive שלך
final_df.to_pickle('/content/drive/My Drive/Colab_Notebooks/data/Full_data_with_finetuned_embeddings.pkl')

print("\n🎉 All clusters fine-tuned and embeddings saved successfully!")
print("✅ Data saved at: /content/drive/My Drive/Colab_Notebooks/data/Full_data_with_finetuned_embeddings.pkl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 

In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Base path for your fine-tuned models
model_base_path = "/content/drive/My Drive/Colab_Notebooks/models/"

# Load the DataFrame containing your data and cluster assignments
df = pd.read_pickle('/content/drive/My Drive/Colab_Notebooks/data/full_clustered.pkl')


# Check if all values in 'F100' are strings
if not all(isinstance(x, str) for x in df['F100']):
    # Delete rows where 'F100' is not a string
    df = df[df['F100'].apply(lambda x: isinstance(x, str))]



# Function to generate embeddings using a fine-tuned model
def get_finetuned_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embedding

# Iterate through clusters and generate embeddings
all_clusters_dfs = []
for cluster_num in df['ag_title_cluster'].unique():
    print(f"\n🚀 Generating Embeddings for Cluster #{cluster_num}")

    # Filter data for the current cluster
    df_cluster = df[df['ag_title_cluster'] == cluster_num].copy()

    # Load the fine-tuned model and tokenizer for this cluster
    model_path = os.path.join(model_base_path, f"fine_tuned_bert_cluster_{cluster_num}")
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertModel.from_pretrained(model_path)

    # Generate embeddings for 'F100' texts in the cluster
    embeddings_list = []
    for idx, text in enumerate(df_cluster['F100']):
        embedding = get_finetuned_embedding(text, model, tokenizer)
        embeddings_list.append(embedding)
        if (idx + 1) % 100 == 0 or (idx + 1) == len(df_cluster):
            print(f"Processed {idx+1}/{len(df_cluster)} embeddings.")

    # Add the embeddings to the DataFrame
    df_cluster['F100_finetuning_embedding'] = embeddings_list
    all_clusters_dfs.append(df_cluster)

# Combine all cluster DataFrames
final_df = pd.concat(all_clusters_dfs, ignore_index=True)

# Save the DataFrame with the embeddings
final_df.to_pickle('/content/drive/My Drive/Colab_Notebooks/data/Full_data_with_finetuned_embeddings.pkl')
print("\n🎉 All F100 embeddings generated and saved successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🚀 Generating Embeddings for Cluster #0


Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/Colab_Notebooks/models/fine_tuned_bert_cluster_0 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed 100/3818 embeddings.
Processed 200/3818 embeddings.
Processed 300/3818 embeddings.
Processed 400/3818 embeddings.
Processed 500/3818 embeddings.
Processed 600/3818 embeddings.
Processed 700/3818 embeddings.
Processed 800/3818 embeddings.
Processed 900/3818 embeddings.
Processed 1000/3818 embeddings.
Processed 1100/3818 embeddings.
Processed 1200/3818 embeddings.
Processed 1300/3818 embeddings.
Processed 1400/3818 embeddings.
Processed 1500/3818 embeddings.
Processed 1600/3818 embeddings.
Processed 1700/3818 embeddings.
Processed 1800/3818 embeddings.
Processed 1900/3818 embeddings.
Processed 2000/3818 embeddings.
Processed 2100/3818 embeddings.
Processed 2200/3818 embeddings.
Processed 2300/3818 embeddings.
Processed 2400/3818 embeddings.
Processed 2500/3818 embeddings.
Processed 2600/3818 embeddings.
Processed 2700/3818 embeddings.
Processed 2800/3818 embeddings.
Processed 2900/3818 embeddings.
Processed 3000/3818 embeddings.
Processed 3100/3818 embeddings.
Processed 3200/38

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/Colab_Notebooks/models/fine_tuned_bert_cluster_2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed 100/1686 embeddings.
Processed 200/1686 embeddings.
Processed 300/1686 embeddings.
Processed 400/1686 embeddings.
Processed 500/1686 embeddings.
Processed 600/1686 embeddings.
Processed 700/1686 embeddings.
Processed 800/1686 embeddings.
Processed 900/1686 embeddings.
Processed 1000/1686 embeddings.
Processed 1100/1686 embeddings.
Processed 1200/1686 embeddings.
Processed 1300/1686 embeddings.
Processed 1400/1686 embeddings.
Processed 1500/1686 embeddings.
Processed 1600/1686 embeddings.
Processed 1686/1686 embeddings.

🚀 Generating Embeddings for Cluster #5


...

Processed 100/2575 embeddings.
Processed 200/2575 embeddings.
Processed 300/2575 embeddings.
Processed 400/2575 embeddings.
Processed 500/2575 embeddings.
Processed 600/2575 embeddings.
Processed 700/2575 embeddings.
Processed 800/2575 embeddings.
Processed 900/2575 embeddings.
Processed 1000/2575 embeddings.
Processed 1100/2575 embeddings.
Processed 1200/2575 embeddings.
Processed 1300/2575 embeddings.
Processed 1400/2575 embeddings.
Processed 1500/2575 embeddings.
Processed 1600/2575 embeddings.
Processed 1700/2575 embeddings.
Processed 1800/2575 embeddings.
Processed 1900/2575 embeddings.
Processed 2000/2575 embeddings.
Processed 2100/2575 embeddings.
Processed 2200/2575 embeddings.
Processed 2300/2575 embeddings.
Processed 2400/2575 embeddings.
Processed 2500/2575 embeddings.
Processed 2575/2575 embeddings.

🚀 Generating Embeddings for Cluster #3


Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/Colab_Notebooks/models/fine_tuned_bert_cluster_3 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed 100/1196 embeddings.
Processed 200/1196 embeddings.
Processed 300/1196 embeddings.
Processed 400/1196 embeddings.
Processed 500/1196 embeddings.
Processed 600/1196 embeddings.
Processed 700/1196 embeddings.
Processed 800/1196 embeddings.
Processed 900/1196 embeddings.
Processed 1000/1196 embeddings.
Processed 1100/1196 embeddings.
Processed 1196/1196 embeddings.

🚀 Generating Embeddings for Cluster #4


Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/Colab_Notebooks/models/fine_tuned_bert_cluster_4 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed 100/3335 embeddings.
Processed 200/3335 embeddings.
Processed 300/3335 embeddings.
Processed 400/3335 embeddings.
Processed 500/3335 embeddings.
Processed 600/3335 embeddings.
Processed 700/3335 embeddings.
Processed 800/3335 embeddings.
Processed 900/3335 embeddings.
Processed 1000/3335 embeddings.
Processed 1100/3335 embeddings.
Processed 1200/3335 embeddings.
Processed 1300/3335 embeddings.
Processed 1400/3335 embeddings.
Processed 1500/3335 embeddings.
Processed 1600/3335 embeddings.
Processed 1700/3335 embeddings.
Processed 1800/3335 embeddings.
Processed 1900/3335 embeddings.
Processed 2000/3335 embeddings.
Processed 2100/3335 embeddings.
Processed 2200/3335 embeddings.
Processed 2300/3335 embeddings.
Processed 2400/3335 embeddings.
Processed 2500/3335 embeddings.
Processed 2600/3335 embeddings.
Processed 2700/3335 embeddings.
Processed 2800/3335 embeddings.
Processed 2900/3335 embeddings.
Processed 3000/3335 embeddings.
Processed 3100/3335 embeddings.
Processed 3200/33

In [5]:
# prompt: now normalize It l2
import numpy as np
from sklearn.preprocessing import normalize

# Assuming 'final_df' is your DataFrame and 'F100_finetuning_embedding' is the column containing the embeddings
# Convert the embeddings to numpy arrays for normalization
embeddings_np = np.array(final_df['F100_finetuning_embedding'].tolist())

# Normalize the embeddings using L2 norm
normalized_embeddings = normalize(embeddings_np, norm='l2')

# Convert the normalized embeddings back to a list of lists
normalized_embeddings_list = normalized_embeddings.tolist()

# Update the DataFrame with the normalized embeddings
final_df['F100_finetuning_embedding_normalized'] = normalized_embeddings_list

# Save the updated DataFrame
final_df.to_pickle('/content/drive/My Drive/Colab_Notebooks/data/New_Normalized_Finetuned.pkl')
print("\n🎉 Embeddings normalized and saved successfully!")



🎉 Embeddings normalized and saved successfully!


##Fine Tuning emb on titles

In [ ]:
##modle on titles
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from google.colab import drive
import os
from sklearn.preprocessing import normalize

drive.mount('/content/drive')

# נתיב לתיקיית המודלים המאומנים
model_base_path = "/content/drive/My Drive/Colab_Notebooks/models/"

# טעינת הנתונים (שכבר כוללים את ה-F100 embeddings)
df = pd.read_pickle('/content/drive/My Drive/Colab_Notebooks/data/full_data_with_finetuned_embeddings.pkl')

# פונקציה להפקת embedding מהמודל שעבר fine-tuning
def get_finetuned_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embedding

# מעבר על כל קלאסטר והפקת embeddings לכותרות
all_clusters_dfs = []

for cluster_num in df['ag_title_cluster'].unique():
    print(f"\n🚀 Extracting Title Embeddings for Cluster #{cluster_num}")

    # סינון הנתונים של הקלאסטר
    cluster_df = df[df['ag_title_cluster'] == cluster_num].copy()

    # בדיקת קיום המודל המאומן
    model_path = os.path.join(model_base_path, f"fine_tuned_bert_cluster_{cluster_num}")

    if not os.path.exists(model_path):
        print(f"🚨 Model for Cluster {cluster_num} not found at {model_path}! Skipping...")
        continue

    # טעינת המודל והטוקנייזר שעברו אימון עבור הקלאסטר הזה
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertModel.from_pretrained(model_path)

    # יצירת רשימת embeddings
    embeddings_list = []
    for idx, text in enumerate(cluster_df['title']):
        embedding = get_finetuned_embedding(text, model, tokenizer)
        embeddings_list.append(embedding)
        if (idx + 1) % 100 == 0 or (idx + 1) == len(cluster_df):
            print(f"Processed {idx+1}/{len(cluster_df)} title embeddings.")

    # שמירת ה-embeddings בעמודה חדשה
    cluster_df['finetuning_title_embedding'] = embeddings_list

    # הוספה לרשימת הקלאסטרים
    all_clusters_dfs.append(cluster_df)

# איחוד כל הקלאסטרים ל-DataFrame אחד
final_df = pd.concat(all_clusters_dfs, ignore_index=True)

# Normalize the embeddings
final_df['normalized_titles_finetuned_embedding'] = df['finetuning_embedding'].apply(lambda x: normalize(x.reshape(1, -1), norm='l2').flatten())

# Save the updated DataFrame
final_df.to_pickle('/content/drive/My Drive/Colab_Notebooks/data/full_data_with_finetuned_embeddings.pkl')
print("\n🎉 All title Normalembeddings extracted successfully!")
print("✅ Data saved at: /content/drive/My Drive/Colab_Notebooks/data/full_data_with_finetuned_embeddings.pkl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🚀 Extracting Title Embeddings for Cluster #0


Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/Colab_Notebooks/models/fine_tuned_bert_cluster_0 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed 100/3818 title embeddings.
Processed 200/3818 title embeddings.
Processed 300/3818 title embeddings.
Processed 400/3818 title embeddings.
Processed 500/3818 title embeddings.
Processed 600/3818 title embeddings.
Processed 700/3818 title embeddings.
Processed 800/3818 title embeddings.
Processed 900/3818 title embeddings.
Processed 1000/3818 title embeddings.
Processed 1100/3818 title embeddings.
Processed 1200/3818 title embeddings.
Processed 1300/3818 title embeddings.
Processed 1400/3818 title embeddings.
Processed 1500/3818 title embeddings.
Processed 1600/3818 title embeddings.
Processed 1700/3818 title embeddings.
Processed 1800/3818 title embeddings.
Processed 1900/3818 title embeddings.
Processed 2000/3818 title embeddings.
Processed 2100/3818 title embeddings.
Processed 2200/3818 title embeddings.
Processed 2300/3818 title embeddings.
Processed 2400/3818 title embeddings.
Processed 2500/3818 title embeddings.
Processed 2600/3818 title embeddings.
Processed 2700/3818 t

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/Colab_Notebooks/models/fine_tuned_bert_cluster_1 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed 100/2575 title embeddings.
Processed 200/2575 title embeddings.
Processed 300/2575 title embeddings.
Processed 400/2575 title embeddings.
Processed 500/2575 title embeddings.
Processed 600/2575 title embeddings.
Processed 700/2575 title embeddings.
Processed 800/2575 title embeddings.
Processed 900/2575 title embeddings.
Processed 1000/2575 title embeddings.
Processed 1100/2575 title embeddings.
Processed 1200/2575 title embeddings.
Processed 1300/2575 title embeddings.
Processed 1400/2575 title embeddings.
Processed 1500/2575 title embeddings.
Processed 1600/2575 title embeddings.
Processed 1700/2575 title embeddings.
Processed 1800/2575 title embeddings.
Processed 1900/2575 title embeddings.
Processed 2000/2575 title embeddings.
Processed 2100/2575 title embeddings.
Processed 2200/2575 title embeddings.
Processed 2300/2575 title embeddings.
Processed 2400/2575 title embeddings.
Processed 2500/2575 title embeddings.
Processed 2575/2575 title embeddings.

🚀 Extracting Title E

...

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/Colab_Notebooks/models/fine_tuned_bert_cluster_4 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed 100/3335 title embeddings.
Processed 200/3335 title embeddings.
Processed 300/3335 title embeddings.
Processed 400/3335 title embeddings.
Processed 500/3335 title embeddings.
Processed 600/3335 title embeddings.
Processed 700/3335 title embeddings.
Processed 800/3335 title embeddings.
Processed 900/3335 title embeddings.
Processed 1000/3335 title embeddings.
Processed 1100/3335 title embeddings.
Processed 1200/3335 title embeddings.
Processed 1300/3335 title embeddings.
Processed 1400/3335 title embeddings.
Processed 1500/3335 title embeddings.
Processed 1600/3335 title embeddings.
Processed 1700/3335 title embeddings.
Processed 1800/3335 title embeddings.
Processed 1900/3335 title embeddings.
Processed 2000/3335 title embeddings.
Processed 2100/3335 title embeddings.
Processed 2200/3335 title embeddings.
Processed 2300/3335 title embeddings.
Processed 2400/3335 title embeddings.
Processed 2500/3335 title embeddings.
Processed 2600/3335 title embeddings.
Processed 2700/3335 t

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/My Drive/Colab_Notebooks/models/fine_tuned_bert_cluster_5 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed 100/2388 title embeddings.
Processed 200/2388 title embeddings.
Processed 300/2388 title embeddings.
Processed 400/2388 title embeddings.
Processed 500/2388 title embeddings.
Processed 600/2388 title embeddings.
Processed 700/2388 title embeddings.
Processed 800/2388 title embeddings.
Processed 900/2388 title embeddings.
Processed 1000/2388 title embeddings.
Processed 1100/2388 title embeddings.
Processed 1200/2388 title embeddings.
Processed 1300/2388 title embeddings.
Processed 1400/2388 title embeddings.
Processed 1500/2388 title embeddings.
Processed 1600/2388 title embeddings.
Processed 1700/2388 title embeddings.
Processed 1800/2388 title embeddings.
Processed 1900/2388 title embeddings.
Processed 2000/2388 title embeddings.
Processed 2100/2388 title embeddings.
Processed 2200/2388 title embeddings.
Processed 2300/2388 title embeddings.
Processed 2388/2388 title embeddings.


NameError: name 'nt' is not defined

Embeddings normalized and saved successfully!


finetuned embedding for titles